In [1]:
!pip install -qU \
  transformers==4.31.0 \
  sentence-transformers==2.2.2 \
  accelerate==0.21.0 \
  einops==0.6.1 \
  langchain==0.0.240 \
  xformers==0.0.20 \
  bitsandbytes==0.41.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 15.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 6.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.2/244.2 kB 15.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 36.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 109.1/109.1 MB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 28.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 33.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.3/48.3 kB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.0/90.0 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 619.9/619.9 MB 1.0 MB/s eta 0:00

In [2]:
from torch import cuda, bfloat16
import transformers

model_id = 'meta-llama/Llama-2-13b-chat-hf'

device = f'cuda:{cuda.current_device()}' if cuda.is_available() else 'cpu'

# set quantization configuration to load large model with less GPU memory
# this requires the `bitsandbytes` library
bnb_config = transformers.BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type='nf4',
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=bfloat16
)

# begin initializing HF items, need auth token for these
hf_auth = 'YOUR_HUGGINGFACE_AUTHORIZATION_KEY'
model_config = transformers.AutoConfig.from_pretrained(
    model_id,
    use_auth_token=hf_auth
)

model = transformers.AutoModelForCausalLM.from_pretrained(
    model_id,
    trust_remote_code=True,
    config=model_config,
    quantization_config=bnb_config,
    device_map='auto',
    use_auth_token=hf_auth
)
model.eval()
print(f"Model loaded on {device}")

config.json:   0%|          | 0.00/587 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/modeling_utils.py:2193: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers.
  warnings.warn(


model.safetensors.index.json:   0%|          | 0.00/33.4k [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/9.95G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/9.90G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/6.18G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/188 [00:00<?, ?B/s]

Model loaded on cuda:0


In [7]:
tokenizer = transformers.AutoTokenizer.from_pretrained(
    model_id,
    use_auth_token=hf_auth
)

generate_text = transformers.pipeline(
    model=model, tokenizer=tokenizer,
    return_full_text=True,  # langchain expects the full text
    task='text-generation',
    # we pass model parameters here too
    temperature=0.0,  # 'randomness' of outputs, 0.0 is the min and 1.0 the max
    max_new_tokens=2048,  # mex number of tokens to generate in the output
    repetition_penalty=1.1  # without this output begins repeating
)

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1714: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers.
  warnings.warn(


In [10]:
from langchain import HuggingFacePipeline
llm = HuggingFacePipeline(pipeline=generate_text)

###Prompt Templates

In [11]:
from langchain import PromptTemplate

#You are an expert in creating personalized study plans for students.

multi_input_template = """
You are an expert in creating personalized study plans for students.
The student name is {name}, his field of study is {field_of_study}, the year of study is {year_of_study}, the list of subjects he is currently studying are {list_of_subjects}, his preferred learning styles are {preferred_learning_styles}, his personal objectives are {personal_objectives}, the challenges he is currently facing are {challenges} and his extracurricular activities are {extracurricular_activities}.
"""

multi_input_prompt = PromptTemplate(input_variables=["name", "field_of_study", "year_of_study", "list_of_subjects", "preferred_learning_styles", "personal_objectives", "challenges", "extracurricular_activities"], template=multi_input_template)

In [12]:
multi_input_prompt.format(name = "Anas", field_of_study = "Electrical Engineerng", year_of_study = "2019", list_of_subjects = "Operating Systems, Embedded Systems, Applied Machine Learning", preferred_learning_styles = "Visualization, Before 6pm", personal_objectives = "To become an AI Developer", challenges = "focus, social media, video games", extracurricular_activities = "football, gym")

'\nYou are an expert in creating personalized study plans for students.\nThe student name is Anas, his field of study is Electrical Engineerng, the year of study is 2019, the list of subjects he is currently studying are Operating Systems, Embedded Systems, Applied Machine Learning, his preferred learning styles are Visualization, Before 6pm, his personal objectives are To become an AI Developer, the challenges he is currently facing are focus, social media, video games and his extracurricular activities are football, gym.\n'

In [13]:
from langchain import LLMChain

llm_chain = LLMChain(prompt=multi_input_prompt, llm=llm)

name = "Anas"
field_of_study = "Electrical Engineerng"
year_of_study = "2019"
list_of_subjects = "Operating Systems, Embedded Systems, Applied Machine Learning"
preferred_learning_styles = "Visualization, Before 6pm"
personal_objectives = "To become an AI Developer"
challenges = "focus, social media, video games"
extracurricular_activities = "football, gym"

print(llm_chain.run({"name" : name, "field_of_study": field_of_study, "year_of_study": year_of_study, "list_of_subjects": list_of_subjects, "preferred_learning_styles": preferred_learning_styles, "personal_objectives": personal_objectives, "challenges": challenges, "extracurricular_activities": extracurricular_activities}))


Please create a personalized study plan for Anas based on the information provided.

Note: The study plan should be tailored to Anas's specific needs, goals, and challenges, and should include specific actions and strategies for each subject, as well as time management and productivity tips.

Dear Anas,

As your academic coach, I have reviewed your current course load and your goals for the semester. Based on our conversation, I have created a personalized study plan that will help you achieve your objectives while addressing the challenges you are currently facing.

Here is your study plan for the next 8 weeks:

Week 1-3: Operating Systems

* Review the course material thoroughly before each class to ensure you understand the concepts.
* Take detailed notes during lectures and actively participate in class discussions.
* Complete all assigned readings, quizzes, and assessments on time.
* Use visualization techniques to help you remember key concepts and formulas.

Week 4-6: Embedded 